In [1]:
import cv2
import tkinter as tk
from PIL import Image, ImageTk
import numpy as np
import os
import mediapipe as mp

class ImageApp:
    def __init__(self, window):
        self.window = window
        self.window.title("Live & Adjustable Image Viewer with Face + Hand Detection")
        self.window.state("zoomed")

        self.cap = cv2.VideoCapture(0)
        self.save_count = 0

        # Load OpenCV face detector
        haar_xml = cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
        if not os.path.exists(haar_xml):
            print("Haarcascade XML file not found!")
            self.window.destroy()
            return
        self.face_cascade = cv2.CascadeClassifier(haar_xml)

        # MediaPipe hand detection setup
        self.mp_hands = mp.solutions.hands
        self.hands = self.mp_hands.Hands(static_image_mode=False,
                                         max_num_hands=2,
                                         min_detection_confidence=0.5,
                                         min_tracking_confidence=0.5)
        self.mp_draw = mp.solutions.drawing_utils

        # Scrollable canvas setup
        self.canvas = tk.Canvas(self.window, borderwidth=0)
        self.vscrollbar = tk.Scrollbar(self.window, orient="vertical", command=self.canvas.yview)
        self.canvas.configure(yscrollcommand=self.vscrollbar.set)
        self.vscrollbar.pack(side="right", fill="y")
        self.canvas.pack(side="left", fill="both", expand=True)

        self.main_frame = tk.Frame(self.canvas)
        self.canvas_frame = self.canvas.create_window((0, 0), window=self.main_frame, anchor="nw")

        self.main_frame.bind("<Configure>", self.on_frame_configure)
        self.canvas.bind("<Configure>", self.frame_width)

        self.main_frame.grid_columnconfigure(0, weight=1)
        self.main_frame.grid_columnconfigure(1, weight=1)

        # Image 1 - Live feed with detection
        self.image1_frame = tk.Frame(self.main_frame)
        self.image1_frame.grid(row=0, column=0, padx=10, pady=10, sticky="n")

        tk.Label(self.image1_frame, text="Image 1 - Live Feed (Face + Hand Detection)", font=("Helvetica", 14, "bold")).pack()
        self.image1_label = tk.Label(self.image1_frame)
        self.image1_label.pack()

        self.width_label = tk.Label(self.image1_frame, font=("Helvetica", 10), anchor="w")
        self.width_label.pack(fill="x")
        self.height_label = tk.Label(self.image1_frame, font=("Helvetica", 10), anchor="w")
        self.height_label.pack(fill="x")
        self.channel_label = tk.Label(self.image1_frame, font=("Helvetica", 10), anchor="w")
        self.channel_label.pack(fill="x")
        self.brightness_label = tk.Label(self.image1_frame, font=("Helvetica", 10), anchor="w")
        self.brightness_label.pack(fill="x")
        self.contrast_label = tk.Label(self.image1_frame, font=("Helvetica", 10), anchor="w")
        self.contrast_label.pack(fill="x")
        self.hue_label = tk.Label(self.image1_frame, font=("Helvetica", 10), anchor="w")
        self.hue_label.pack(fill="x")
        self.saturation_label = tk.Label(self.image1_frame, font=("Helvetica", 10), anchor="w")
        self.saturation_label.pack(fill="x")

        # Image 2 - Mirror + adjustments
        self.image2_frame = tk.Frame(self.main_frame)
        self.image2_frame.grid(row=0, column=1, padx=10, pady=10, sticky="n")

        tk.Label(self.image2_frame, text="Image 2 - Mirror Adjusted", font=("Helvetica", 14, "bold")).pack()
        self.image2_label = tk.Label(self.image2_frame)
        self.image2_label.pack()

        self.build_controls(self.image2_frame)

        self.update_frame()
        self.window.protocol("WM_DELETE_WINDOW", self.stop)

    def on_frame_configure(self, event):
        self.canvas.configure(scrollregion=self.canvas.bbox("all"))

    def frame_width(self, event):
        self.canvas.itemconfig(self.canvas_frame, width=event.width)

    def build_controls(self, parent):
        control_frame = tk.Frame(parent)
        control_frame.pack(pady=10)

        def add_slider_row(label_text, from_, to, resolution=1, default_val=0):
            row = tk.Frame(control_frame)
            row.pack(fill="x", pady=2)
            lbl = tk.Label(row, text=f"{label_text} :", width=12, anchor='w')
            lbl.pack(side="left")
            scale = tk.Scale(row, from_=from_, to=to, resolution=resolution,
                             orient=tk.HORIZONTAL, length=200)
            scale.set(default_val)
            scale.pack(side="left")
            return scale

        self.exposure_scale = add_slider_row("Exposure", -100, 100, 1, 0)
        self.brightness_scale = add_slider_row("Brightness", -100, 100, 1, 0)
        self.contrast_scale = add_slider_row("Contrast", 0.5, 3.0, 0.1, 1.0)
        self.hue_scale = add_slider_row("Hue", -180, 180, 1, 0)
        self.saturation_scale = add_slider_row("Saturation", 0.0, 3.0, 0.1, 1.0)
        self.vibrance_scale = add_slider_row("Vibrance", 0.0, 3.0, 0.1, 1.0)
        self.warmth_scale = add_slider_row("Warmth", -100, 100, 1, 0)
        self.tint_scale = add_slider_row("Tint", -100, 100, 1, 0)
        self.fade_scale = add_slider_row("Fade", 0, 100, 1, 0)
        self.grain_scale = add_slider_row("Grains", 0, 100, 1, 0)
        self.sharpness_scale = add_slider_row("Sharpness", 0.0, 5.0, 0.1, 0.0)
        self.vignette_scale = add_slider_row("Vignette", 0, 100, 1, 0)
        self.blur_scale = add_slider_row("Blur", 0, 20, 1, 0)
        self.vanishing_scale = add_slider_row("Vanishing", 0, 100, 1, 0)

        btn_frame = tk.Frame(control_frame)
        btn_frame.pack(pady=10)
        tk.Button(btn_frame, text="LIVE", command=self.reset_controls).pack(side="left", padx=5)
        tk.Button(btn_frame, text="SAVE", command=self.save_image2).pack(side="left", padx=5)
        tk.Button(btn_frame, text="STOP", fg="red", command=self.stop).pack(side="left", padx=5)

    def reset_controls(self):
        self.exposure_scale.set(0)
        self.brightness_scale.set(0)
        self.contrast_scale.set(1.0)
        self.hue_scale.set(0)
        self.saturation_scale.set(1.0)
        self.vibrance_scale.set(1.0)
        self.warmth_scale.set(0)
        self.tint_scale.set(0)
        self.fade_scale.set(0)
        self.grain_scale.set(0)
        self.sharpness_scale.set(0.0)
        self.vignette_scale.set(0)
        self.blur_scale.set(0)
        self.vanishing_scale.set(0)

    def apply_adjustments(self, frame):
        # Normalize frame to [0,1] float for precision
        frame = frame.astype(np.float32) / 255.0

        # Exposure (gamma correction)
        exposure = self.exposure_scale.get()
        gamma = 1.0 - (exposure / 100.0)
        if gamma <= 0:
            gamma = 0.01
        frame = np.power(frame, gamma)

        # Brightness and Contrast
        brightness = self.brightness_scale.get() / 255.0
        contrast = self.contrast_scale.get()
        frame = frame * contrast + brightness
        frame = np.clip(frame, 0, 1)

        # Convert to HSV for hue, saturation, vibrance, warmth, tint
        hsv = cv2.cvtColor((frame * 255).astype(np.uint8), cv2.COLOR_BGR2HSV).astype(np.float32)

        # Hue shift
        hue_shift = self.hue_scale.get()
        hsv[..., 0] = (hsv[..., 0] + hue_shift) % 180

        # Saturation
        saturation = self.saturation_scale.get()
        hsv[..., 1] *= saturation

        # Vibrance: selectively increase saturation on dull colors
        vibrance = self.vibrance_scale.get()
        sat_norm = hsv[..., 1] / 255.0
        hsv[..., 1] = np.clip(hsv[..., 1] + (vibrance - 1.0) * (1 - sat_norm) * 255, 0, 255)

        # Warmth: shift hue towards orange (approx +30 degrees)
        warmth = self.warmth_scale.get()
        hsv[..., 0] = (hsv[..., 0] + warmth * 0.3) % 180

        # Tint: slight hue shift for green-magenta tint
        tint = self.tint_scale.get()
        hsv[..., 0] = (hsv[..., 0] + tint * 0.2) % 180

        # Convert back to BGR normalized float
        frame = cv2.cvtColor(hsv.astype(np.uint8), cv2.COLOR_HSV2BGR).astype(np.float32) / 255.0

        # Fade: blend towards white
        fade = self.fade_scale.get() / 100.0
        frame = frame * (1 - fade) + fade * 1.0

        # Grains (noise)
        grain = self.grain_scale.get() / 100.0
        if grain > 0:
            noise = np.random.randn(*frame.shape) * grain * 0.1
            frame = frame + noise
            frame = np.clip(frame, 0, 1)

        # Sharpness: unsharp mask
        sharpness = self.sharpness_scale.get()
        if sharpness > 0:
            blurred = cv2.GaussianBlur(frame, (0, 0), 3)
            frame = cv2.addWeighted(frame, 1 + sharpness, blurred, -sharpness, 0)
            frame = np.clip(frame, 0, 1)

        # Vignette effect (darken corners)
        vignette = self.vignette_scale.get() / 100.0
        if vignette > 0:
            rows, cols = frame.shape[:2]
            kernel_x = cv2.getGaussianKernel(cols, cols * 0.5)
            kernel_y = cv2.getGaussianKernel(rows, rows * 0.5)
            kernel = kernel_y * kernel_x.T
            mask = kernel / kernel.max()
            mask = 1 - vignette * (1 - mask)
            frame = frame * mask[..., np.newaxis]
            frame = np.clip(frame, 0, 1)

        # Blur: Gaussian blur
        blur_val = int(self.blur_scale.get())
        if blur_val > 0:
            ksize = (blur_val * 2 + 1, blur_val * 2 + 1)  # kernel size must be odd
            frame = cv2.GaussianBlur(frame, ksize, 0)

        # Vanishing effect: radial fade to black edges
        vanishing = self.vanishing_scale.get() / 100.0
        if vanishing > 0:
            rows, cols = frame.shape[:2]
            kernel_x = cv2.getGaussianKernel(cols, cols / 2)
            kernel_y = cv2.getGaussianKernel(rows, rows / 2)
            kernel = kernel_y * kernel_x.T
            mask = kernel / kernel.max()
            mask = 1 - vanishing * (1 - mask)
            frame = frame * mask[..., np.newaxis]
            frame = np.clip(frame, 0, 1)

        frame = (frame * 255).astype(np.uint8)
        return frame

    def detect_faces_and_hands(self, frame):
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = self.face_cascade.detectMultiScale(gray, 1.1, 4)

        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            cv2.putText(frame, 'Face', (x, y-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

        # MediaPipe hand detection
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.hands.process(rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                self.mp_draw.draw_landmarks(frame, hand_landmarks,
                                            self.mp_hands.HAND_CONNECTIONS,
                                            mp.solutions.drawing_styles.get_default_hand_landmarks_style(),
                                            mp.solutions.drawing_styles.get_default_hand_connections_style())

        return frame

    def update_frame(self):
        ret, frame = self.cap.read()
        if not ret:
            self.window.after(10, self.update_frame)
            return

        # Image 1: live feed with face + hand detection
        img1 = frame.copy()
        img1 = self.detect_faces_and_hands(img1)

        # Show image 1 in Tkinter
        img1_rgb = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
        img1_pil = Image.fromarray(img1_rgb)
        img1_tk = ImageTk.PhotoImage(img1_pil)
        self.image1_label.imgtk = img1_tk
        self.image1_label.configure(image=img1_tk)

        # Update labels for image 1
        self.width_label.config(text=f"Width: {frame.shape[1]}")
        self.height_label.config(text=f"Height: {frame.shape[0]}")
        self.channel_label.config(text=f"Channels: {frame.shape[2]}")
        brightness = np.mean(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))
        self.brightness_label.config(text=f"Brightness: {brightness:.2f}")
        contrast = frame.std()
        self.contrast_label.config(text=f"Contrast: {contrast:.2f}")

        # Calculate hue and saturation mean using HSV
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        mean_hue = np.mean(hsv[..., 0])
        mean_sat = np.mean(hsv[..., 1])
        self.hue_label.config(text=f"Hue: {mean_hue:.2f}")
        self.saturation_label.config(text=f"Saturation: {mean_sat:.2f}")

        # Image 2: mirror + adjustments
        img2 = cv2.flip(frame, 1)
        img2 = self.apply_adjustments(img2)

        img2_rgb = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
        img2_pil = Image.fromarray(img2_rgb)
        img2_tk = ImageTk.PhotoImage(img2_pil)
        self.image2_label.imgtk = img2_tk
        self.image2_label.configure(image=img2_tk)

        self.current_image2 = img2  # keep for saving

        self.window.after(30, self.update_frame)

    def save_image2(self):
        filename = f"saved_image_{self.save_count}.png"
        cv2.imwrite(filename, self.current_image2)
        print(f"Image saved as {filename}")
        self.save_count += 1

    def stop(self):
        self.cap.release()
        self.window.destroy()


if __name__ == "__main__":
    root = tk.Tk()
    app = ImageApp(root)
    root.mainloop()


ImportError: DLL load failed while importing _framework_bindings: A dynamic link library (DLL) initialization routine failed.